In [1]:
from tensorflow.keras.layers  import Dense, Conv2D, Dropout, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score,precision_score,recall_score
from sklearn.metrics import ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import numpy as np
import glob
import cv2
import zipfile

In [2]:
!mkdir ~/.kaggle
!touch ~/.kaggle/kaggle.json

api_token = {"username":"danielreyesdhats","key":"546f8b8a22d19b7001b860532f3a07c1"}

import json

with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)

!chmod 600 ~/.kaggle/kaggle.json

In [3]:
!kaggle datasets download -d grassknoted/asl-alphabet

In [4]:
local_zip = "/kaggle/working/asl-alphabet.zip"
zip_ref = zipfile.ZipFile(local_zip, "r")
file_extract = "/kaggle/working/asl-alphabet"
zip_ref.extractall(file_extract) 
zip_ref.close()

In [5]:
features = []
labels = []

classes = ["A","B","C","D","E","F","G","H","I","J","K","L","M","N","O","P","Q","R","S","T","U","V","W","X","Y","Z", 'del','nothing','space']
n_classes=len(classes)
for i in range(len(classes)):
    file_list = glob.glob("/kaggle/working/asl-alphabet/asl_alphabet_train/asl_alphabet_train/" + classes[i] + "/*.jpg")
    for item in range(800):
        features.append(file_list[item])
        labels.append([i])
print("Dataset Feature size : ",len(features))
print("Dataset labels size : ",len(labels))

In [6]:
features,labels = shuffle(features,labels,random_state=0)

In [7]:
data = []
for ima in features:
  data.append(cv2.imread(ima))

In [8]:
X_train, X_test, y_train, y_test = train_test_split(data, labels, train_size=0.7)
print("Train data : ",len(X_train),len(y_train))
print("Validation data : ",len(X_test),len(y_test))

In [9]:
input_dim = X_train[0].shape
input_dim

In [10]:
model = Sequential()
model.add(Conv2D(filters=64, kernel_size=(4,4), activation='relu', input_shape=input_dim))
model.add(Conv2D(64, kernel_size=(4,4), strides=2, activation='relu'))
model.add(Dropout(0.5))
model.add(Conv2D(128, kernel_size=(4,4), strides=1, activation='relu'))
model.add(Conv2D(128, kernel_size=(4,4), strides=2, activation='relu'))
model.add(Dropout(0.5))
model.add(Conv2D(256, kernel_size=(4,4), strides=1, activation='relu'))
model.add(Conv2D(256, kernel_size=(4,4), strides=2, activation='relu'))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(n_classes, activation='softmax'))

opt = Adam(learning_rate=0.0001)
model.compile(optimizer=opt,loss="sparse_categorical_crossentropy",metrics=['accuracy'])
model.summary()

In [11]:
X_traina = np.array(X_train)
y_traina = np.array(y_train) 
X_testa = np.array(X_test)
y_testa = np.array(y_test)

In [12]:
del data, X_train, X_test, y_train, y_test

In [13]:
history = model.fit(X_traina,
          y_traina,
          epochs=10,
          validation_data = (X_testa,y_testa))

In [14]:
plt.style.use('seaborn')
plt.figure(figsize=(10,10))
plt.plot(history.history['loss'], color='b', label="Training loss")
plt.plot(history.history['val_loss'], color='r', label="Validation loss")
plt.legend()
plt.show()

plt.figure()

plt.figure(figsize=(8,8))
plt.plot(history.history['accuracy'], color='b', label="Training accuracy")
plt.plot(history.history['val_accuracy'], color='r',label="Validation accuracy")
plt.legend()
plt.show()

In [16]:
y_pred = np.argmax(model.predict(X_testa),1)
print("Precision : {:.2f} %".format(precision_score(y_pred,y_testa,average='macro')))
print("Recall    : {:.2f} %".format(recall_score(y_pred,y_testa,average='macro')))
print("F1 Score  : {:.2f} %".format(f1_score(y_pred,y_testa,average='macro')))

In [17]:
model.save('model.h5')

In [ ]:
!pip install tensorflowjs

In [20]:
!mkdir carpeta_salida

In [21]:
!tensorflowjs_converter --input_format keras model.h5 carpeta_salida

In [30]:
X_testa[0].shape

In [38]:
examp = np.expand_dims(X_testa[0], axis=0)
examp.shape

In [40]:
np.argmax(model.predict(examp,1))